This Jupyter Notebook contains the first project for Information Retrieval 1 taught at the UvA. Code is made by Oscar Ligthart, Nicole Ferreira Silverio and Arend van Dormalen.

ANSWER TO THEORETICAL QUESTION 1A

The chance of a type 1 error ($\alpha$) increases each time an experiment is repeated, if it's not corrected. The new $\alpha$ for _m_ experiments is $1 − (1 − \alpha)^m ≈ m\alpha$. This is called the Family-wise error rate.

ANSWER TO THEORETICAL QUESTION 1B,

The chance of a type 1 error needs to be corrected. This can be done by the Bonferroni correction, which rejects the null hypothesis for any $p_i$ when it it lower than or equal to $\frac{\alpha}{m}$.
      

ANSWER TO THEORETICAL QUESTION 2

Assume two ranked lists created by two different rankers. List $l1$ contains documents $d1$, $d2$ and $d3$ in that order. List $l2$ contains documents $d2$, $d3$ and $d4$ in that order. Now assume that the only relevant document is $d3$, which will therefore be clicked on most often. From our judgment, it is obvious that $l2$ is the most relevant list as it has placed $d3$ on a higher position. However, in Team Draft Interleaving, these algorithms will be evaluated as having equal performance.

In this situation, $d3$ will always be the third item on the interleaved list. After the first coin flip, $d2$ will be removed from $l1$ as this document has already been supplied by $l2$.  At the second coin flip, $d3$ will be the next document for both lists. This causes the relevance for both lists to be the same, as they now both have the same chance of supplying the only relevant document to the interleaved list.

Step 1:

The following cell represents the creation of all possible relevance pairs E and P

In [ ]:
import itertools
import numpy as np
import random
import re
import matplotlib.pyplot as plt
import statistics
import scipy.stats

# first get the sequence options
relevance = ['N', 'R', 'HR']
options = list(itertools.product(relevance, repeat = 5))

# create all possible pairs in sequence options
pair_index = list(itertools.permutations(range(len(options)), 2))


pairs = []

for temp_pair in pair_index:
    pairs.append([options[temp_pair[0]], options[temp_pair[1]]])    


# Offline measures

Each cell represents one of the following three offline measures: Average precision, nDCG, ERR

In [ ]:
def get_average_precision(ranking):
    rel = 0
    AP_numerator = 0
    # get amount of relevant documents
    for i, doc in enumerate(ranking):
        if doc == 'R' or doc == 'HR':
            rel += 1
            AP_numerator += rel/(i+1)
            
    return rel, AP_numerator

# new dict for average precision for both P and E (key is pair, value is average precisions))
AP_delta = {}

# new dict for delta measures (AP, nDCG and ERR will be stored here per pair)
delta_values = {}

# get precision for all pairs
for pair in pairs:
    # first calculate numerator for average precision for P
    P = pair[0]    
    P_rel, P_AP_numerator = get_average_precision(P)
 
    # now calculate numerator for average precision for E
    E = pair[1]
    E_rel, E_AP_numerator = get_average_precision(E)

    # get total number of relevant documents returned from query
    total_rel = P_rel + E_rel
    
    # calculate average precision for both P and E
    P_AP = P_AP_numerator/total_rel
    E_AP = E_AP_numerator/total_rel
    
    # store results in a dict
    AP_delta[(P,E)] = E_AP - P_AP
    
    # store AP delta measures in list format in dict
    delta_values[(P,E)] = [E_AP - P_AP]


In [ ]:
##### nDCG cell #####
def get_nDCG(ranking):
    DCG = 0
    
    # loop through ranking
    for i, rank in enumerate(ranking):
        # decide what the relative rank is
        if rank == 'HR':
            rel_r = 2
        elif rank == 'R':
            rel_r = 1
        elif rank == 'N':
            rel_r = 0
        
        DCG += (2**rel_r - 1)/(np.log2(1+(i+1)))
    
    return DCG

# new dict for average precision for both P and E (key is pair, value is average precisions))
nDCG_delta = {}

counter = 0
same_counter = 0
lower_counter = 0

# get nDCG for all pairs
for pair in pairs:    
    # first for P
    P = pair[0]
    P_DCG = get_nDCG(P)
    
    # then for E
    E = pair[1]
    E_DCG = get_nDCG(E)
    
    nDCG_delta[(P,E)] = E_DCG - P_DCG
    
    # add nDCG delta measure to dict
    delta_values[(P,E)].append(E_DCG - P_DCG)



In [ ]:
##### ERR cell #####

def get_ERR(ranking):
    
    ERR = 0
    p = 1
    max_rel = 2
    
    # loop through ranking
    for i, rank in enumerate(ranking):        
        # start at second rank
        if i != 0:            
            # decide what the relative rank is
            if rank == 'HR':
                rel_r = 2
            elif rank == 'R':
                rel_r = 1
            elif rank == 'N':
                rel_r = 0

            # Calculate R with the mapping function
            R = (2**rel_r - 1)/(2**max_rel)

            # Modify ERR value
            ERR += p * (R/i)

            # Modify p
            p = p*(1-R)
    
    return ERR

# new dict for ERR values for both P and E (key is pair, value is ERR value))
ERR_delta = {}

# get ERR for all pairs
for pair in pairs:
    
    # first for P
    P = pair[0]
    P_ERR = get_ERR(P)
    
    # then for E
    E = pair[1]
    E_ERR = get_ERR(E)
    
    ERR_delta[(P,E)] = E_ERR - P_ERR
    
    # add ERR delta measures to dict
    delta_values[(P,E)].append(E_ERR -  P_ERR)
    


## Filtering and interleaving pairs
In the following cells the pairs that hold delta values that are positive are extracted for further use. We extracted the pairs holding a positive delta value for all offline measures. Moreover, we took the mean of delta values from all offline measures and extracted the pairs that had a positive delta value for this mean.
Afterwards, we used balance interleaving to create one ranking out of both algorithms.

In [ ]:
##### Filter pairs #####

def filter_all_pairs(pairs_dict):
    # this function is used to extract pairs that have a positive delta value for the mean of all offline measures
    pairs_list = []
    dict_items = pairs_dict.items()
    for pair in dict_items:
        scores = pair[1]
        avg = sum(scores, 0.0)/len(scores)
        if avg > 0.0:
            pairs_list.append(pair[0])
    
    return pairs_list

def filter_metric_pairs(pairs_dict):
    # this function is used to extract pairs that have a positive delta value based on offline measures
    pairs_list = []
    dict_items = pairs_dict.items()
    for pair in dict_items:
        scores = pair[1]
        if scores > 0.0:
            pairs_list.append(pair[0])
    
    return pairs_list


# Iterate through pairs
def interleaving(pairs):
    all_results = []
    all_origins = []
    for pair in pairs:

        # Flip a coin, assign winning and losing
        # P = pair[0], E = pair[1]
        coin_winner = random.randint(0,1)
        winner = pair[coin_winner]
        loser = pair[1 - coin_winner]

        # initiate lists
        resulting_list = []
        origin_list = []

        # iterate through lists, fill up results and origin list
        for i in range(len(winner)):
            resulting_list.append(winner[i])
            origin_list.append(coin_winner)
            resulting_list.append(loser[i])
            origin_list.append(1-coin_winner)

        all_results.append(resulting_list)
        all_origins.append(origin_list)
    
    return all_results, all_origins

In [ ]:
##### Balanced Interleaving #####

print("Before:", len(delta_values.items()))

all_pairs = filter_all_pairs(delta_values)

print("After:", len(all_pairs))

all_results, all_origins = interleaving(all_pairs)

# now get pairs for which delta measure is positive for every offline metric
AP_pairs = filter_metric_pairs(AP_delta)
DCG_pairs = filter_metric_pairs(nDCG_delta)
ERR_pairs = filter_metric_pairs(ERR_delta)

# interleave the pairs for which E outperforms P per offline metric
AP_results, AP_origins = interleaving(AP_pairs)
DCG_results, DCG_origins = interleaving(DCG_pairs)
ERR_results, ERR_origins = interleaving(ERR_pairs)

print(len(all_results))

# Random Click Model
The following cells represent the random click model and its simulation. Simulation is run for A) every offline measure for which E outperforms P and B) the mean delta values of all offline measures for which E outperforms P.

In [ ]:
##### Random Click Model #####

# Learns parameter
def learn_param_RCM(data):
    
    # open file and read
    lines=data.readlines()

    clicks = 0
    documents = 0

    # Acquire total amount of queries and clicks
    for line in lines:
        items = re.split(r'\t+',line)
        if items[2] == "Q":
            # Per query 10 documents are shown
            documents += 10
        elif items[2] == "C":
            clicks += 1
    
    # Calculate rho
    rho = clicks/documents
    
    return rho

# Predicts a click probability
def predict_prob_RCM(ranking, param):
    
    # get the click probability for every document in ranking
    click_prob = []
    for doc in ranking:
        click_prob.append(param)
        
    return click_prob

# Decide whether document is clicked on
def click_doc_RCM(click_prob):
    clicked = []
    for prob in click_prob:
        chance = random.random()
        if chance <= prob:
            clicked.append(1)
        else:
            clicked.append(0)
    return clicked

def RCM_simulation(pairs, origins, rho):

    N = 50
    p_RCM_list = []

    # simulate experiment N times
    for i in range(N):
        # keep track of which algorithm won
        E_win = 0
        P_win = 0
        # loop through all rankings
        for j, ranking in enumerate(pairs):

            # predict probability of clicking
            click_prob = predict_prob_RCM(ranking, rho)

            # get which documents were clicked
            clicked = click_doc_RCM(click_prob)

            # now shuffle the origin list so documents are picked at random
            origin_shuffle = random.sample(origins[j], len(origins[j]))

            E_click = 0
            P_click = 0
            # check whether the clicked documents were produced by E or P
            for h, click in enumerate(clicked):
                if click == 1 and origin_shuffle[h] == 1:
                    E_click += 1
                elif click == 1 and origin_shuffle[h] == 0:
                    P_click += 1

            # determine whether E or P won
            if E_click > P_click:
                E_win += 1
            elif P_click > E_click:
                P_win += 1

        # proportion of times E won
        p = E_win / (E_win + P_win)
        p_RCM_list.append(p)
        
    return p_RCM_list


In [ ]:
###### Simulate random click model ######

# get parameter out of data
f=open("YandexRelPredChallenge.txt","r")
rho = learn_param_RCM(f)
f.close()

# get the p for the average of all metrics
p_RCM_list = RCM_simulation(all_results, all_origins, rho)

# get the p for every metric 
p_AP_RCM_list = RCM_simulation(AP_results, AP_origins, rho)
p_DCG_RCM_list = RCM_simulation(DCG_results, DCG_origins, rho)
p_ERR_RCM_list = RCM_simulation(ERR_results, ERR_origins, rho)

print(p_RCM_list)
print(p_AP_RCM_list)
print(p_DCG_RCM_list)
print(p_ERR_RCM_list)



# Simple Dynamic Bayesian Model
The following cells represent the simiple dynamic bayesian model and its simulation. Simulation is run for A) every offline measure for which E outperforms P and B) the mean delta values of all offline measures for which E outperforms P.

In [ ]:
##### Simple Dynamic Bayesian Model #####

# Learns parameter
def learn_param_DBM(file):
    
    lines = file.readlines()

    #previous_session = 0 # Keep track of session number to determine if click is last click.
    previous_type = ""
    
    clicks = 0
    #last_clicks_session = 0
    last_clicks_query = 0

    lines.reverse() # Reversed order, so it is detectable if a click is last.
    for line in lines:
        items = re.split(r'\t+',line) #strip tabs
        #current_session = items[0]
        current_type = items[2]
        #if current_type == "C" and current_session != previous_session:
            #last_clicks_session += 1
        if current_type == "C" and previous_type == "Q": 
            last_clicks_query += 1
        if current_type == "C":
            clicks += 1
        #previous_session = current_session
        previous_type = current_type

    sigma = last_clicks_query/clicks
    
    return sigma
        

# Predicts a click probability
def predict_prob_DBM(rank, sigma):
    # for the click probability, we'll need P(A) and P(E)

    # first get alpha, which will be set according to the level of relevance of a document
    if rank == 'HR':
        alpha = 0.9
    elif rank == 'R':
        alpha = 0.3
    elif rank == 'N':
        alpha = 0 
    
    # check if user will click on the document (depending on alpha)
    x = random.random()
    if x <= alpha:
        P_A = 1
    else:
        P_A = 0
            
    # since we are using a simple DBM, gamma will always be one    
    gamma = 1
    
    return P_A, gamma  
       
        
# Decide which documents are clicked
def click_doc_DBM(ranking, sigma):
    # this function takes a ranking list and a value for the parameter sigma as input and uses
    # these to determine which documents in the ranking list are clicked on
    
    # set P(E) to 1 (first snippet is always read)
    P_E = 1
    
    clicked = []
    
    # run through the ranking to decide whether a document will be clicked or not
    for rank in ranking:
        P_A, gamma = predict_prob_DBM(rank, sigma)
        
        # based on probability, set click to 1 or 0
        if P_A == 1 and P_E == 1:
            P_C = 1
        else:
            P_C = 0
     
        clicked.append(P_C)
        
        # if user has clicked, check if user is satisfied
        if P_C == 1:
            # now check if user is satisfied
            x = random.random()
            if x <= sigma:
                # if satisfied, user will not read any more snippets (thus click nothing)
                P_E = 0
            else:
                # if user is not satisfied, user will read next snippet (thus possibly click)
                P_E = 1 
        
    return clicked       

def DBM_simulation(pairs, origins, sigma):
    N = 50
    p_DBM_list = []
    # simulate experiment N times
    for i in range(N):

        # keep track of which algorithm won
        E_win = 0
        P_win = 0
        # loop through all rankings
        for j, ranking in enumerate(pairs):

            # get which documents were clicked
            clicked = click_doc_DBM(ranking, sigma)

            E_click = 0
            P_click = 0

            current_origin = origins[j]

            # check whether the clicked documents were produced by E or P
            for h, click in enumerate(clicked):
                if click == 1 and current_origin[h] == 1:
                    E_click += 1
                elif click == 1 and current_origin[h] == 0:
                    P_click += 1

            # determine whether E or P won
            if E_click > P_click:
                E_win += 1
            elif P_click > E_click:
                P_win += 1
        #print(E_win, P_win)
        # proportion of times E won
        p = E_win / (E_win + P_win)
        p_DBM_list.append(p)
    return p_DBM_list

In [ ]:
##### Simulate dynamic bayesian model #####

# get parameter out of data
f=open("YandexRelPredChallenge.txt","r")
sigma = learn_param_DBM(f)
f.close()

print(len(all_results))
# get the p for the average of all metrics
p_DBM_list = DBM_simulation(all_results, all_origins, sigma)


# get the p for every metric 
p_AP_DBM_list = DBM_simulation(AP_results, AP_origins, sigma)
p_DCG_DBM_list = DBM_simulation(DCG_results, DCG_origins, sigma)
p_ERR_DBM_list = DBM_simulation(ERR_results, ERR_origins, sigma)

print(p_DBM_list)
print(p_AP_DBM_list)
print(p_DCG_DBM_list)
print(p_ERR_DBM_list)


# Analysis

In [ ]:
# Effect of different values for delta on 
# create new dict with all values for which delta is positive
pairs_dict = {}
for key, value in delta_values.items():
    avg = sum(value, 0.0)/len(value)
    if avg > 0.0:
        pairs_dict[key] = avg

keys = sorted(pairs_dict, key=pairs_dict.get)

index = round(len(pairs_dict)/10)
# take first 10 % of the sorted dictionary to represent the low delta values
low_deltas = keys[:index]

# take last 10 % of the sorted dictionary to represent the high delta values
high_deltas = keys[-index:]

# interleave both pair lists
low_delta_results, low_delta_origins = interleaving(low_deltas)
high_delta_results, high_delta_origins = interleaving(high_deltas)

# get rho out of data
f=open("YandexRelPredChallenge.txt","r")
rho = learn_param_RCM(f)
f.close()

# get sigma out of data                     
f=open("YandexRelPredChallenge.txt","r")
sigma = learn_param_DBM(f)
f.close()

# run the random click model on both delta's
p_RCM_ld = RCM_simulation(low_delta_results, low_delta_origins, rho)
p_RCM_hd = RCM_simulation(high_delta_results, high_delta_origins, rho)

# run the dynamic bayesion model on both delta's
p_DBM_ld = DBM_simulation(low_delta_results, low_delta_origins, sigma)
p_DBM_hd = DBM_simulation(high_delta_results, high_delta_origins, sigma)

# run statistical tests to find significant differences
scipy.stats.ttest_ind(p_DBM_ld, p_DBM_hd)

####### CREATE PLOT ########

multiple_bars = plt.figure()

x = [0.25,0.8]
names = ['RCM', 'DBM']

bars_RCM = [sum(p_RCM_ld)/len(p_RCM_ld), sum(p_DBM_ld)/len(p_DBM_ld)]
bars_DBM = [sum(p_RCM_hd)/len(p_RCM_hd), sum(p_DBM_hd)/len(p_DBM_hd)]

ax = plt.subplot(111)

ax.bar(x[0]-0.1, bars_RCM[0] ,width=0.2,color='b',align='center')
ax.bar(x[0]+0.1, bars_RCM[1] ,width=0.2,color='g',align='center')

ax.bar(x[1]-0.1, bars_DBM[0] ,width=0.2,color='b',align='center')
ax.bar(x[1]+0.1, bars_DBM[1] ,width=0.2,color='g',align='center')
ax.set_xticks([x[0],x[1]])
ax.set_xticklabels(names)

plt.show()



In [ ]:
##### PLOT OVER THE AVERAGE OF ALL METRICS #####

# average over all metrics and standard deviation for random click model
avg_ALL_RCM = statistics.mean(p_RCM_list)
stdev_ALL_RCM = statistics.pstdev(p_RCM_list)

# average over all metrics and standard deviation for dynamic bayesian model
avg_ALL_DBM = statistics.mean(p_DBM_list)
stdev_ALL_DBM = statistics.pstdev(p_DBM_list)

# plot bar graph over all metrics, RCM compared to DBM
ax = plt.gca()

x = range(1, 3)
y = (avg_ALL_RCM, avg_ALL_DBM)
z = (stdev_ALL_RCM, stdev_ALL_DBM)
names = ('RCM', 'DBM')
clr = ('r', 'b')
plt.bar(x, y, width=0.5, color = clr, yerr = z, align="center")

ind = range(1, 3)
plt.xticks(ind, x)
ax.set_xticklabels(names)
plt.show()

In [ ]:
##### PLOTS FOR EACH METRIC #####

# average and standard deviation per metric for random click model
avg_AP_RCM = statistics.mean(p_AP_RCM_list)
stdev_AP_RCM = statistics.pstdev(p_AP_RCM_list)

avg_DCG_RCM = statistics.mean(p_DCG_RCM_list)
stdev_DCG_RCM = statistics.pstdev(p_DCG_RCM_list)

avg_ERR_RCM = statistics.mean(p_ERR_RCM_list)
stdev_ERR_RCM = statistics.pstdev(p_ERR_RCM_list)

# average and standard deviation per metric for random click model
avg_AP_DBM = statistics.mean(p_AP_DBM_list)
stdev_AP_DBM = statistics.pstdev(p_AP_DBM_list)

avg_DCG_DBM = statistics.mean(p_DCG_DBM_list)
stdev_DCG_DBM = statistics.pstdev(p_DCG_DBM_list)

avg_ERR_DBM = statistics.mean(p_ERR_DBM_list)
stdev_ERR_DBM = statistics.pstdev(p_ERR_DBM_list)


# plot bar graph over all metrics, RCM compared to DBM
ax = plt.gca()

x = range(1, 7)
y = (avg_AP_RCM, avg_DCG_RCM, avg_ERR_RCM, avg_AP_DBM, avg_DCG_DBM, avg_ERR_DBM)
z = (stdev_AP_RCM, stdev_DCG_RCM, stdev_ERR_RCM, stdev_AP_DBM, stdev_DCG_DBM, stdev_ERR_DBM)
names = ('AP', 'DCG', 'ERR', 'AP', 'DCG', 'ERR')
clr = ('r', 'r', 'r', 'b', 'b', 'b')
plt.bar(x, y, width=0.5, color = clr, yerr = z, align="center")

ind = range(1, 7)
plt.xticks(ind, x)
ax.set_xticklabels(names)
plt.show()

In [ ]:
##### Check percentages of overlapping pairs between offline measures #####,

print("AP:",len(AP_pairs))
print("DCG:",len(DCG_pairs))
print("ERR:",len(ERR_pairs))

print("AP+DCG:", (1- (len(set(AP_pairs+DCG_pairs))/(len(AP_pairs)+ len(DCG_pairs))))*2)
print("AP+ERR:", (1- (len(set(AP_pairs+ERR_pairs))/(len(AP_pairs)+ len(ERR_pairs))))*2)
print("DCG+ERR:", (1- (len(set(DCG_pairs+ERR_pairs))/(len(DCG_pairs)+ len(ERR_pairs))))*2)
print("AP+DCG+ERR:", (1- (len(set(AP_pairs+DCG_pairs+ERR_pairs))/(len(AP_pairs)+len(DCG_pairs)+len(ERR_pairs))))*2)

print("AP in DCG:", len(list(set(AP_pairs).intersection(DCG_pairs)))/len(AP_pairs))
print("AP in ERR:", len(list(set(AP_pairs).intersection(ERR_pairs)))/len(AP_pairs))
print("DCG in AP:", len(list(set(DCG_pairs).intersection(AP_pairs)))/len(DCG_pairs))
print("DCG in ERR:", len(list(set(DCG_pairs).intersection(ERR_pairs)))/len(DCG_pairs))
print("ERR in AP:", len(list(set(ERR_pairs).intersection(AP_pairs)))/len(ERR_pairs))
print("ERR in DCG:", len(list(set(ERR_pairs).intersection(DCG_pairs)))/len(ERR_pairs))
